# Rețele generative

Rețelele Neuronale Recurente (RNN) și variantele lor cu celule cu porți, cum ar fi Celulele cu Memorie pe Termen Lung (LSTM) și Unitățile Recurente cu Porți (GRU), au oferit un mecanism pentru modelarea limbajului, adică pot învăța ordonarea cuvintelor și pot oferi predicții pentru următorul cuvânt dintr-o secvență. Acest lucru ne permite să folosim RNN-urile pentru **sarcini generative**, cum ar fi generarea obișnuită de text, traducerea automată și chiar generarea de descrieri pentru imagini.

În arhitectura RNN discutată în unitatea anterioară, fiecare unitate RNN producea următoarea stare ascunsă ca ieșire. Totuși, putem adăuga și o altă ieșire fiecărei unități recurente, ceea ce ne-ar permite să generăm o **secvență** (care este egală ca lungime cu secvența originală). Mai mult, putem folosi unități RNN care nu acceptă o intrare la fiecare pas, ci doar primesc un vector de stare inițială și apoi produc o secvență de ieșiri.

În acest notebook, ne vom concentra pe modele generative simple care ne ajută să generăm text. Pentru simplitate, să construim o **rețea la nivel de caractere**, care generează text literă cu literă. În timpul antrenării, trebuie să luăm un corpus de text și să-l împărțim în secvențe de litere.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Construirea vocabularului de caractere

Pentru a construi o rețea generativă la nivel de caractere, trebuie să împărțim textul în caractere individuale, în loc de cuvinte. Acest lucru poate fi realizat prin definirea unui tokenizator diferit:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Să vedem exemplul de cum putem codifica textul din setul nostru de date:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Antrenarea unui RNN generativ

Modul în care vom antrena RNN pentru a genera text este următorul. La fiecare pas, vom lua o secvență de caractere de lungime `nchars` și vom cere rețelei să genereze următorul caracter de ieșire pentru fiecare caracter de intrare:

![Imagine care arată un exemplu de generare RNN a cuvântului 'HELLO'.](../../../../../translated_images/ro/rnn-generate.56c54afb52f9781d.webp)

În funcție de scenariul concret, este posibil să dorim să includem și câteva caractere speciale, cum ar fi *sfârșit-de-secvență* `<eos>`. În cazul nostru, dorim doar să antrenăm rețeaua pentru generarea continuă de text, așa că vom fixa dimensiunea fiecărei secvențe să fie egală cu `nchars` tokeni. Prin urmare, fiecare exemplu de antrenament va consta din `nchars` intrări și `nchars` ieșiri (care sunt secvența de intrare deplasată cu un simbol spre stânga). Un minibatch va consta din mai multe astfel de secvențe.

Modul în care vom genera minibatch-uri este să luăm fiecare text de știri de lungime `l` și să generăm toate combinațiile posibile de intrare-ieșire din acesta (vor exista `l-nchars` astfel de combinații). Acestea vor forma un minibatch, iar dimensiunea minibatch-urilor va fi diferită la fiecare pas de antrenament.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Acum să definim rețeaua generatorului. Aceasta poate fi bazată pe orice celulă recurentă despre care am discutat în unitatea anterioară (simplă, LSTM sau GRU). În exemplul nostru vom folosi LSTM.

Deoarece rețeaua primește caractere ca intrare, iar dimensiunea vocabularului este destul de mică, nu avem nevoie de un strat de încorporare (embedding), intrarea codificată one-hot poate fi trimisă direct către celula LSTM. Totuși, deoarece transmitem numerele caracterelor ca intrare, trebuie să le codificăm one-hot înainte de a le trimite către LSTM. Acest lucru se face apelând funcția `one_hot` în timpul trecerii `forward`. Codificatorul de ieșire va fi un strat liniar care va converti starea ascunsă într-o ieșire codificată one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

În timpul antrenamentului, dorim să putem eșantiona text generat. Pentru a face acest lucru, vom defini funcția `generate`, care va produce un șir de caractere de lungime `size`, începând de la șirul inițial `start`.

Modul în care funcționează este următorul. Mai întâi, vom trece întregul șir `start` prin rețea și vom obține starea de ieșire `s` și următorul caracter prezis `out`. Deoarece `out` este codificat one-hot, folosim `argmax` pentru a obține indexul caracterului `nc` din vocabular și utilizăm `itos` pentru a determina caracterul real, pe care îl adăugăm la lista rezultată de caractere `chars`. Acest proces de generare a unui caracter este repetat de `size` ori pentru a genera numărul necesar de caractere.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Acum să trecem la antrenament! Bucla de antrenament este aproape aceeași ca în toate exemplele noastre anterioare, dar în loc să afișăm acuratețea, generăm și afișăm text eșantionat la fiecare 1000 de epoci.

O atenție specială trebuie acordată modului în care calculăm pierderea. Trebuie să calculăm pierderea având ca intrare ieșirea codificată one-hot `out` și textul așteptat `text_out`, care este lista indicilor caracterelor. Din fericire, funcția `cross_entropy` așteaptă ca prim argument ieșirea ne-normalizată a rețelei și ca al doilea argument numărul clasei, ceea ce este exact ceea ce avem. De asemenea, efectuează automat media pe dimensiunea minibatch-ului.

De asemenea, limităm antrenamentul la un număr de mostre definit de `samples_to_train`, pentru a nu aștepta prea mult. Vă încurajăm să experimentați și să încercați antrenamente mai lungi, posibil pentru mai multe epoci (în acest caz, ar trebui să creați o altă buclă în jurul acestui cod).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Acest exemplu generează deja un text destul de bun, dar poate fi îmbunătățit în mai multe moduri:

* **Generarea mai bună a minibatch-urilor**. Modul în care am pregătit datele pentru antrenament a fost să generăm un minibatch dintr-un singur eșantion. Acest lucru nu este ideal, deoarece minibatch-urile au dimensiuni diferite, iar unele dintre ele nici măcar nu pot fi generate, deoarece textul este mai mic decât `nchars`. De asemenea, minibatch-urile mici nu utilizează suficient GPU-ul. Ar fi mai înțelept să luăm un bloc mare de text din toate eșantioanele, să generăm toate perechile input-output, să le amestecăm și să generăm minibatch-uri de dimensiuni egale.

* **LSTM multilayer**. Are sens să încercăm 2 sau 3 straturi de celule LSTM. Așa cum am menționat în unitatea anterioară, fiecare strat de LSTM extrage anumite tipare din text, iar în cazul unui generator la nivel de caractere, ne putem aștepta ca nivelul inferior al LSTM să fie responsabil pentru extragerea silabelor, iar nivelurile superioare - pentru cuvinte și combinații de cuvinte. Acest lucru poate fi implementat simplu prin transmiterea unui parametru pentru numărul de straturi către constructorul LSTM.

* De asemenea, ai putea să experimentezi cu **unități GRU** și să vezi care dintre ele oferă performanțe mai bune, precum și cu **dimensiuni diferite ale straturilor ascunse**. Un strat ascuns prea mare poate duce la overfitting (de exemplu, rețeaua va învăța textul exact), iar o dimensiune mai mică s-ar putea să nu producă un rezultat bun.


## Generarea textului moale și temperatura

În definiția anterioară a funcției `generate`, alegeam întotdeauna caracterul cu cea mai mare probabilitate ca următorul caracter în textul generat. Acest lucru ducea adesea la faptul că textul "cicla" între aceleași secvențe de caractere din nou și din nou, ca în acest exemplu:
```
today of the second the company and a second the company ...
```

Totuși, dacă ne uităm la distribuția probabilităților pentru următorul caracter, este posibil ca diferența dintre câteva dintre cele mai mari probabilități să nu fie semnificativă, de exemplu, un caracter poate avea probabilitatea 0.2, iar altul 0.19, etc. De exemplu, când căutăm următorul caracter în secvența '*play*', următorul caracter poate fi la fel de bine un spațiu sau **e** (ca în cuvântul *player*).

Acest lucru ne conduce la concluzia că nu este întotdeauna "corect" să selectăm caracterul cu probabilitatea cea mai mare, deoarece alegerea celui de-al doilea cel mai probabil caracter poate duce, de asemenea, la un text semnificativ. Este mai înțelept să **eșantionăm** caracterele din distribuția probabilităților oferită de rezultatul rețelei.

Această eșantionare poate fi realizată folosind funcția `multinomial`, care implementează așa-numita **distribuție multinomială**. O funcție care implementează această generare de text **moale** este definită mai jos:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Am introdus încă un parametru numit **temperatură**, care este utilizat pentru a indica cât de strict ar trebui să ne bazăm pe cea mai mare probabilitate. Dacă temperatura este 1.0, efectuăm o eșantionare multinomială echitabilă, iar când temperatura crește spre infinit - toate probabilitățile devin egale și selectăm aleator următorul caracter. În exemplul de mai jos putem observa că textul devine lipsit de sens atunci când creștem temperatura prea mult și seamănă cu un text "ciclic" generat rigid atunci când se apropie de 0.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
